## Noise Resilience Benchmarking of Hybrid Quantum-Classical Face Verification Under NISQ Constraints​


### Step 1：Image Filtering - from preprocess.py

In [36]:
# --- Imports & config ---
import os, random
from pathlib import Path

import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18

SEED = 123
random.seed(SEED); torch.manual_seed(SEED)

ROOT = Path("./")
DATA_A = ROOT / "data" / "Positive"
DATA_B = ROOT / "data" / "Negative"


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

train_root = ROOT / "_bin_dataset" / "train"
val_root   = ROOT / "_bin_dataset" / "val"

def build_split(srcA, srcB, train_ratio=0.8, limit_per_class=300):
    import shutil
    for p in [train_root, val_root]:
        if p.exists():
            shutil.rmtree(p)
    for p in [train_root/srcA.name, train_root/srcB.name, val_root/srcA.name, val_root/srcB.name]:
        p.mkdir(parents=True, exist_ok=True)

    def list_imgs(d: Path):
        exts = {".jpg",".jpeg",".png",".bmp",".webp"}
        return [p for p in d.rglob("*") if p.suffix.lower() in exts]

    A = list_imgs(srcA)
    B = list_imgs(srcB)
    random.shuffle(A); random.shuffle(B)
    A = A[:min(limit_per_class, len(A))]
    B = B[:min(limit_per_class, len(B))]

    kA = int(len(A)*train_ratio)
    kB = int(len(B)*train_ratio)

    for src in A[:kA]: shutil.copy(src, train_root/srcA.name/src.name)
    for src in A[kA:]: shutil.copy(src, val_root/srcA.name/src.name)
    for src in B[:kB]: shutil.copy(src, train_root/srcB.name/src.name)
    for src in B[kB:]: shutil.copy(src, val_root/srcB.name/src.name)

build_split(DATA_A, DATA_B)

train_ds = datasets.ImageFolder(train_root, transform=tfm)
val_ds   = datasets.ImageFolder(val_root,   transform=tfm)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=0)

print("Classes:", train_ds.classes)


Classes: ['Negative', 'Positive']


In [ ]:
ckpt_backbone = ROOT / "outputs" / "resnet18_backbone_only.pt"
if not ckpt_backbone.exists():
    alt = ROOT / "resnet18_finetuned.pt"
    assert alt.exists(), ""
    ckpt_backbone = alt

backbone = resnet18(weights=None)
state = torch.load(ckpt_backbone, map_location="cpu")
if isinstance(state, dict) and "state_dict" in state:
    state = state["state_dict"]
_ = backbone.load_state_dict(state, strict=False)

in_feats = backbone.fc.in_features  # 512
backbone.fc = nn.Identity()
backbone.to(device).eval()
for p in backbone.parameters():
    p.requires_grad_(False)


## L512-4

In [ ]:
class L512to4(nn.Module):
    def __init__(self, in_dim=512, hidden_dim=4):
        super().__init__()
        self.fc = nn.Linear(in_dim, hidden_dim)  # W: [4×512], b: [4]
        self.act = nn.Tanh()

    def forward(self, z):  # z: [B,512]
        return self.act(self.fc(z))  # [B,4]

proj = L512to4(in_dim=in_feats, hidden_dim=4).to(device)


## Quantum Circuit

In [ ]:
import pennylane as qml
from pennylane import numpy as pnp

n_qubits = 4
n_layers = 6

# Noise-Free
#dev = qml.device("default.qubit", wires=n_qubits)
# Noise case
dev = qml.device("default.mixed", wires=n_qubits)

def entangle_ladder():

    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])

dev = qml.device("default.mixed", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def quantum_block(x, weights,
                  noise_type="none",
                  gamma_T1=0.0, 
                  gamma_T2=0.0,
                  epsilon=0.0):
    """
    noise_type: "none", "T1", "T2", "rotation"
    gamma_T1: amplitude damping parameter (0~1)
    gamma_T2: dephasing parameter (0~1)
    epsilon: rotation over-rotation (radians)
    """

    # --------------------------
    #   Helper: inject noise
    # --------------------------
    def apply_noise():
        if noise_type == "T1":
            for q in range(n_qubits):
                qml.AmplitudeDamping(gamma_T1, wires=q)

        elif noise_type == "T2":
            for q in range(n_qubits):
                qml.PhaseDamping(gamma_T2, wires=q)

        elif noise_type == "T12":
            for q in range(n_qubits):
                qml.AmplitudeDamping(gamma_T1, wires=q)
                qml.PhaseDamping(gamma_T2, wires=q)

    for q in range(n_qubits):
        qml.Hadamard(wires=q)
        qml.RY(np.pi * x[q] / 2, wires=q)

    # initial layer noise
    apply_noise()

    for l in range(n_layers):

        for q in range(n_qubits):

            if noise_type == "rotation":
                qml.RY(weights[l, q] + epsilon, wires=q)
            else:
                qml.RY(weights[l, q], wires=q)

        # layer noise
        apply_noise()
        entangle_ladder()

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]



In [ ]:
#############################################
# QuantumLayer: input [B,4] → output [B,4]
#############################################
class QuantumLayer(nn.Module):
    def __init__(self, noise_type="none", T1=0.0, T2=0.0, epsilon=0.0):
        super().__init__()
        self.weights = nn.Parameter(0.01 * torch.randn(n_layers, n_qubits))

        self.noise_type = noise_type
        self.gamma_T1 = T1
        self.gamma_T2 = T2
        self.epsilon = epsilon


    def forward(self, x4_batch):
        outs = []
        for i in range(x4_batch.shape[0]):
            y = quantum_block(
                    x4_batch[i],
                    self.weights,
                    noise_type=self.noise_type,
                    gamma_T1=self.gamma_T1,
                    gamma_T2=self.gamma_T2,
                    epsilon=self.epsilon
            )
            y = torch.stack(y)
            outs.append(y)

        return torch.stack(outs).float()


# q_layer = QuantumLayer(noise_type='None').to(device)
# q_layer = QuantumLayer(noise_type="T1", T1=0.02).to(device)
q_layer = QuantumLayer(noise_type="T2", T2=0.049).to(device)
# q_layer = QuantumLayer(noise_type="rotation", epsilon=0.03).to(device)
# q_layer = QuantumLayer(noise_type="T12", T1=0.02, T2=0.049).to(device)


## L4-2

In [ ]:
class L4to2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(4, 2)
    def forward(self, z4):
        return self.fc(z4)

head = L4to2().to(device)

## L+Q+L

In [ ]:
#############################################
# Hybrid model = backbone → L512→4 → Q → L4→2
#############################################
class HybridModel(nn.Module):
    def __init__(self, backbone, proj, q_layer, head):
        super().__init__()
        self.backbone = backbone
        self.proj = proj
        self.q_layer = q_layer
        self.head = head

    def forward(self, imgs):
        with torch.no_grad():
            z512 = self.backbone(imgs)

        x4 = self.proj(z512)            # [B,4]
        zq = self.q_layer(x4)           # [B,4], quantum output
        logits = self.head(zq)          # [B,2]

        return logits

model = HybridModel(backbone, proj, q_layer, head).to(device)

In [ ]:
#############################################
# Training
#############################################
crit = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam([
    {"params": proj.parameters(), "lr": 1e-3},
    {"params": q_layer.parameters(), "lr": 1e-2},
    {"params": head.parameters(), "lr": 1e-3},
])

def run_epoch(loader, train=True):
    model.train(train)

    loss_sum = 0
    correct = 0
    total = 0

    for imgs, labels in loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        if train:
            optimizer.zero_grad()

        logits = model(imgs)  # backbone → L512→4 → Q → L4→2

        loss = crit(logits, labels)

        if train:
            loss.backward()
            optimizer.step()

        # 统计
        loss_sum += loss.item() * imgs.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)

    return loss_sum/total, correct/total

# ----------------- Run training -----------------
for ep in range(1, 6):
    trL, trA = run_epoch(train_loader, True)
    vaL, vaA = run_epoch(val_loader, False)
    print(f"[{ep}] Train {trL:.4f}/{trA:.3f} | Val {vaL:.4f}/{vaA:.3f}")


[1] Train 0.3837/0.929 | Val 0.2991/0.983
[2] Train 0.2882/0.975 | Val 0.2796/0.975
[3] Train 0.2638/0.979 | Val 0.2608/0.975
[4] Train 0.2423/0.983 | Val 0.2418/0.975
[5] Train 0.2291/0.981 | Val 0.2303/0.975


## Test part

In [ ]:
from PIL import Image
import torch
import torch.nn.functional as F

def predict_image(model, image_path, device="cpu"):
    img = Image.open(image_path).convert("RGB")
    img_t = tfm(img).unsqueeze(0).to(device)   # shape [1,3,224,224]
    model.eval()
    with torch.no_grad():
        logits = model(img_t)                 # [1,2]
        probs = F.softmax(logits, dim=1)[0]   # [2]
        pred  = torch.argmax(probs).item()

    class_names = train_ds.classes  # [Negative, Positive]
    
    return {
        "pred_class": class_names[pred],
        "prob_negative": float(probs[0]),
        "prob_positive": float(probs[1]),
        "is_taylor": class_names[pred] == "Positive",
    }

test_img = ROOT / "test1" / "7.jpg"

result = predict_image(model, test_img, device)
result


{'pred_class': 'Positive',
 'prob_negative': 0.18856589496135712,
 'prob_positive': 0.8114340901374817,
 'is_taylor': True}

In [45]:
# T2
from torchvision import datasets
from torch.utils.data import DataLoader
import numpy as np

final_root = ROOT / "final_test_full"

# ImageFolder will use the subfolder names as class labels
final_ds = datasets.ImageFolder(final_root, transform=tfm)
final_loader = DataLoader(final_ds, batch_size=64, shuffle=False, num_workers=0)

print("Final-test classes:", final_ds.classes)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in final_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        # Hybrid model: backbone → L512→4 → QuantumLayer(with noise) → L4→2
        logits = model(imgs)
        preds = logits.argmax(dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Overall accuracy
overall_acc = (all_preds == all_labels).mean()
print(f"\n[final_test] Overall accuracy: {overall_acc:.4f}")

# Per-class accuracy (for 'negative' and 'positive')
for idx, name in enumerate(final_ds.classes):
    mask = (all_labels == idx)
    if mask.sum() == 0:
        continue
    cls_acc = (all_preds[mask] == idx).mean()
    print(f"Class '{name}': acc = {cls_acc:.4f}  (n = {mask.sum()})")


Final-test classes: ['negative', 'positive']

[final_test] Overall accuracy: 0.9235
Class 'negative': acc = 0.9955  (n = 220)
Class 'positive': acc = 0.7917  (n = 120)


In [ ]:
# T1
from torchvision import datasets
from torch.utils.data import DataLoader
import numpy as np

final_root = ROOT / "final_test_full"

# ImageFolder will use the subfolder names as class labels
final_ds = datasets.ImageFolder(final_root, transform=tfm)
final_loader = DataLoader(final_ds, batch_size=64, shuffle=False, num_workers=0)

print("Final-test classes:", final_ds.classes)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in final_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        # Hybrid model: backbone → L512→4 → QuantumLayer(with noise) → L4→2
        logits = model(imgs)
        preds = logits.argmax(dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Overall accuracy
overall_acc = (all_preds == all_labels).mean()
print(f"\n[final_test] Overall accuracy: {overall_acc:.4f}")

# Per-class accuracy (for 'negative' and 'positive')
for idx, name in enumerate(final_ds.classes):
    mask = (all_labels == idx)
    if mask.sum() == 0:
        continue
    cls_acc = (all_preds[mask] == idx).mean()
    print(f"Class '{name}': acc = {cls_acc:.4f}  (n = {mask.sum()})")


Final-test classes: ['negative', 'positive']

[final_test] Overall accuracy: 0.9324
Class 'negative': acc = 0.9864  (n = 220)
Class 'positive': acc = 0.8333  (n = 120)


In [25]:
# Rot
from torchvision import datasets
from torch.utils.data import DataLoader
import numpy as np

final_root = ROOT / "final_test_full"

# ImageFolder will use the subfolder names as class labels
final_ds = datasets.ImageFolder(final_root, transform=tfm)
final_loader = DataLoader(final_ds, batch_size=64, shuffle=False, num_workers=0)

print("Final-test classes:", final_ds.classes)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in final_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        # Hybrid model: backbone → L512→4 → QuantumLayer(with noise) → L4→2
        logits = model(imgs)
        preds = logits.argmax(dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Overall accuracy
overall_acc = (all_preds == all_labels).mean()
print(f"\n[final_test] Overall accuracy: {overall_acc:.4f}")

# Per-class accuracy (for 'negative' and 'positive')
for idx, name in enumerate(final_ds.classes):
    mask = (all_labels == idx)
    if mask.sum() == 0:
        continue
    cls_acc = (all_preds[mask] == idx).mean()
    print(f"Class '{name}': acc = {cls_acc:.4f}  (n = {mask.sum()})")


Final-test classes: ['negative', 'positive']

[final_test] Overall accuracy: 0.9294
Class 'negative': acc = 0.9818  (n = 220)
Class 'positive': acc = 0.8333  (n = 120)


In [22]:
# T12
from torchvision import datasets
from torch.utils.data import DataLoader
import numpy as np

final_root = ROOT / "final_test_full"

# ImageFolder will use the subfolder names as class labels
final_ds = datasets.ImageFolder(final_root, transform=tfm)
final_loader = DataLoader(final_ds, batch_size=64, shuffle=False, num_workers=0)

print("Final-test classes:", final_ds.classes)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in final_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        # Hybrid model: backbone → L512→4 → QuantumLayer(with noise) → L4→2
        logits = model(imgs)
        preds = logits.argmax(dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Overall accuracy
overall_acc = (all_preds == all_labels).mean()
print(f"\n[final_test] Overall accuracy: {overall_acc:.4f}")

# Per-class accuracy (for 'negative' and 'positive')
for idx, name in enumerate(final_ds.classes):
    mask = (all_labels == idx)
    if mask.sum() == 0:
        continue
    cls_acc = (all_preds[mask] == idx).mean()
    print(f"Class '{name}': acc = {cls_acc:.4f}  (n = {mask.sum()})")


Final-test classes: ['negative', 'positive']

[final_test] Overall accuracy: 0.9235
Class 'negative': acc = 0.9955  (n = 220)
Class 'positive': acc = 0.7917  (n = 120)


In [46]:
from pathlib import Path

CKPT_DIR = ROOT / "outputs" / "noise_models"
CKPT_DIR.mkdir(parents=True, exist_ok=True)

# Give it a meaningful name, e.g. include noise type
ckpt_path = CKPT_DIR / "hybrid_noise_model_T2.pt"
print("Checkpoint path:", ckpt_path)

Checkpoint path: outputs\noise_models\hybrid_noise_model_T2.pt


In [ ]:
# Cell: save the trained noisy HybridModel

import torch

# If you tracked best_epoch / best_val_acc in training, you can also include them
save_obj = {
    "model_state": model.state_dict(),
    # Optional extra info:
    # "epoch": best_epoch,
    # "val_acc": best_val_acc,
    # "noise_type": q_layer.noise_type,
}

torch.save(save_obj, ckpt_path)
print(f"Saved noisy model to {ckpt_path}")

Saved noisy model to outputs\noise_models\hybrid_noise_model_T2.pt


In [12]:
from pathlib import Path

CKPT_DIR = ROOT / "outputs" / "noise_models"
CKPT_DIR.mkdir(parents=True, exist_ok=True)

# Give it a meaningful name, e.g. include noise type
ckpt_path = CKPT_DIR / "hybrid_noise_model_T1.pt"
print("Checkpoint path:", ckpt_path)

Checkpoint path: outputs\noise_models\hybrid_noise_model_T1.pt


In [ ]:
# Cell: save the trained noisy HybridModel

import torch

# If you tracked best_epoch / best_val_acc in training, you can also include them
save_obj = {
    "model_state": model.state_dict(),
    # Optional extra info:
    # "epoch": best_epoch,
    # "val_acc": best_val_acc,
    # "noise_type": q_layer.noise_type,
}

torch.save(save_obj, ckpt_path)
print(f"Saved noisy model to {ckpt_path}")

Saved noisy model to outputs\noise_models\hybrid_noise_model_T1.pt


In [26]:
from pathlib import Path

CKPT_DIR = ROOT / "outputs" / "noise_models"
CKPT_DIR.mkdir(parents=True, exist_ok=True)

# Give it a meaningful name, e.g. include noise type
ckpt_path = CKPT_DIR / "hybrid_noise_model_rot.pt"
print("Checkpoint path:", ckpt_path)

Checkpoint path: outputs\noise_models\hybrid_noise_model_rot.pt


In [ ]:
# Cell: save the trained noisy HybridModel

import torch

# If you tracked best_epoch / best_val_acc in training, you can also include them
save_obj = {
    "model_state": model.state_dict(),
    # Optional extra info:
    # "epoch": best_epoch,
    # "val_acc": best_val_acc,
    # "noise_type": q_layer.noise_type,
}

torch.save(save_obj, ckpt_path)
print(f"Saved noisy model to {ckpt_path}")

Saved noisy model to outputs\noise_models\hybrid_noise_model_rot.pt


In [23]:
from pathlib import Path

CKPT_DIR = ROOT / "outputs" / "noise_models"
CKPT_DIR.mkdir(parents=True, exist_ok=True)

# Give it a meaningful name, e.g. include noise type
ckpt_path = CKPT_DIR / "hybrid_noise_model_T12.pt"
print("Checkpoint path:", ckpt_path)

Checkpoint path: outputs\noise_models\hybrid_noise_model_T12.pt


In [ ]:
# Cell: save the trained noisy HybridModel

import torch

# If you tracked best_epoch / best_val_acc in training, you can also include them
save_obj = {
    "model_state": model.state_dict(),
    # Optional extra info:
    # "epoch": best_epoch,
    # "val_acc": best_val_acc,
    # "noise_type": q_layer.noise_type,
}

torch.save(save_obj, ckpt_path)
print(f"Saved noisy model to {ckpt_path}")

Saved noisy model to outputs\noise_models\hybrid_noise_model_T12.pt


In [ ]:
from pathlib import Path

CKPT_DIR = ROOT / "outputs" / "noise_models"
CKPT_DIR.mkdir(parents=True, exist_ok=True)

# Give it a meaningful name, e.g. include noise type
ckpt_path = CKPT_DIR / "hybrid_noise_model_T12r.pt"
print("Checkpoint path:", ckpt_path)

In [ ]:
# Cell: save the trained noisy HybridModel

import torch

# If you tracked best_epoch / best_val_acc in training, you can also include them
save_obj = {
    "model_state": model.state_dict(),
    # Optional extra info:
    # "epoch": best_epoch,
    # "val_acc": best_val_acc,
    # "noise_type": q_layer.noise_type,
}

torch.save(save_obj, ckpt_path)
print(f"Saved noisy model to {ckpt_path}")